In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

In [2]:
spark = SparkSession.builder \
    .appName('parquet_spark_setting') \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

spark

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/12 10:01:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import datetime as dt
from pyspark.sql import functions as F

# =========================
# 1) 오늘 날짜 기준 -> 직전 분기 + 그 분기의 마지막 달 계산
# =========================
today = dt.date.today()

cur_q = (today.month - 1) // 3 + 1          # 1~4
prev_q = cur_q - 1
prev_q_year = today.year
if prev_q == 0:
    prev_q = 4
    prev_q_year -= 1

# 직전 분기의 마지막 달: Q1=3, Q2=6, Q3=9, Q4=12
prev_q_last_month = prev_q * 3

print(f"[AUTO] today={today} / prev_q={prev_q_year}Q{prev_q} / last_month={prev_q_last_month:02d}")

# =========================
# 2) 파티션 경로 생성
# =========================
BUILDING_BASE = "/opt/spark/data/buildingLeader/parquet"
TOJI_BASE     = "/opt/spark/data/tojiSoyuJeongbo/parquet"
REST_BASE     = "/opt/spark/data/restaurant/parquet"
    
building_path = f"{BUILDING_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
toji_path     = f"{TOJI_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
rest_path     = f"{REST_BASE}/dataset={prev_q_year}Q{prev_q}"


print("[AUTO] building_path:", building_path)
print("[AUTO] toji_path    :", toji_path)
print("[AUTO] rest_path    :", rest_path)

# =========================
# 3) 로드 + 지역 필터 (너가 하던대로)
# =========================
building_df = (
    spark.read
    .option("mergeSchema", "false")
    .parquet(building_path)
)
building_gg_df = (
    building_df
    .filter(F.col("region") == "경기")
    .filter(F.col("시군구_코드") == "41461")
)

toji_df = (
    spark.read
    .option("mergeSchema", "false")
    .parquet(toji_path)
)
toji_gg_df = (
    toji_df
    .filter(F.col("region") == "경기")
    .filter(F.col("법정동코드").startswith("41461"))
)

restaurant_df = (
    spark.read
    .option("mergeSchema", "false")
    .parquet(rest_path)
)
restaurant_gg_df = (
    restaurant_df
    .filter(F.col("region") == "경기")
    .filter(F.col("법정동코드").startswith("41461"))
)

print("[AUTO] building_gg_df:", building_gg_df.count())
print("[AUTO] toji_gg_df    :", toji_gg_df.count())
print("[AUTO] restaurant_gg_df:", restaurant_gg_df.count())


[AUTO] today=2026-02-12 / prev_q=2025Q4 / last_month=12
[AUTO] building_path: /opt/spark/data/buildingLeader/parquet/year=2025/month=12
[AUTO] toji_path    : /opt/spark/data/tojiSoyuJeongbo/parquet/year=2025/month=12
[AUTO] rest_path    : /opt/spark/data/restaurant/parquet/dataset=2025Q4


[AUTO] building_gg_df: 51701
[AUTO] toji_gg_df    : 569885
[AUTO] restaurant_gg_df: 14626


**Building Pruning**

In [4]:
building_gg_df = building_gg_df.withColumn(
    "고유번호",
    F.concat(
        F.col("시군구_코드"),        # 5
        F.col("법정동_코드"),        # 5
        F.when(F.col("대지_구분_코드") == "0", F.lit("1"))  # 대지 -> 1
         .when(F.col("대지_구분_코드") == "1", F.lit("2"))  # 산   -> 2
         .otherwise(F.col("대지_구분_코드")),
        F.lpad(F.col("번"), 4, "0"), # 본번
        F.lpad(F.col("지"), 4, "0")  # 부번
    )
)

building_gg_df = (
    building_gg_df
    .filter(F.col("대장_구분_코드") == "1")
    .select(
        F.col("관리_건축물대장_PK"),
        F.col("고유번호"),
        F.col("옥외_자주식_면적(㎡)").alias("옥외자주식면적")
    )
)

building_gg_df.printSchema()
print(building_gg_df.count())
building_gg_df.show(5, truncate=False)

root
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 고유번호: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)

46979
+------------------+-------------------+--------------+
|관리_건축물대장_PK|고유번호           |옥외자주식면적|
+------------------+-------------------+--------------+
|111615014         |4146125322103440000|0             |
|111616678         |4146125323105250000|0             |
|1116112168        |4146126226100420000|0             |
|111619732         |4146135027104580000|0             |
|111618003         |4146135028102150001|0             |
+------------------+-------------------+--------------+
only showing top 5 rows


**Toji Pruning**

In [9]:
toji_gg_df = (
    toji_gg_df
    # 1) 조건 필터
    .filter(F.col("공유인수") == 0)
    .filter(F.col("소유구분코드") == "01")
    # 2) 필요한 컬럼만 선택
    .select(
        F.col("고유번호").cast("string"),
        F.col("법정동명"),
        F.col("지번"),
        F.col("소유권변동원인"),
        F.col("소유권변동일자"),
        F.col("토지면적"),
        F.col("지목"),
        F.col("공시지가")
    )

    # 3) 날짜 파싱
    .withColumn(
        "소유권변동일자_dt",
        F.to_date(F.col("소유권변동일자"), "yyyy-MM-dd")
    )

    # 4) 고유번호별 최신 1행
    .withColumn(
        "rn",
        F.row_number().over(
            Window.partitionBy("고유번호")
                  .orderBy(F.col("소유권변동일자_dt").desc_nulls_last())
        )
    )
    .filter(F.col("rn") == 1)
    .drop("rn", "소유권변동일자_dt")

    # 5) 지번 분리
    .withColumn(
        "본번",
        F.split(F.col("지번"), "-").getItem(0)
    )
    .withColumn(
        "부번",
        F.when(
            F.size(F.split(F.col("지번"), "-")) > 1,
            F.split(F.col("지번"), "-").getItem(1)
        ).otherwise(F.lit(None))
    )
)

# 확인
toji_gg_df.printSchema()
print(toji_gg_df.count())
toji_gg_df.show(10, truncate=False)

root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동원인: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 공시지가: long (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)



111744


[Stage 25:===========================================>              (3 + 1) / 4]

+-------------------+-----------------------------+-----+--------------+--------------+--------+----+--------+----+----+
|고유번호           |법정동명                     |지번 |소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|
+-------------------+-----------------------------+-----+--------------+--------------+--------+----+--------+----+----+
|4146110100100080001|경기도 용인시 처인구 김량장동|8-1  |성명(명칭)변경|2024-09-05    |31.41   |대  |2564000 |8   |1   |
|4146110100100080004|경기도 용인시 처인구 김량장동|8-4  |소유권이전    |2003-03-17    |394.0   |대  |2408000 |8   |4   |
|4146110100100080011|경기도 용인시 처인구 김량장동|8-11 |소유권이전    |2025-07-14    |48.38   |대  |1495000 |8   |11  |
|4146110100100080013|경기도 용인시 처인구 김량장동|8-13 |소유권이전    |2004-08-16    |55.0    |답  |1950000 |8   |13  |
|4146110100100200017|경기도 용인시 처인구 김량장동|20-17|소유권이전    |2016-09-22    |44.0    |대  |2199000 |20  |17  |
|4146110100100220003|경기도 용인시 처인구 김량장동|22-3 |주소변경      |2002-06-24    |29.0    |도로|679400  |22  |3   |
|4146110100100310008|경기도 용인시 처인구 김량장동|31-8 |소유권이전    |2016-02-01

**Restaurant Pruning**

In [10]:
restaurant_gg_df = (
    restaurant_gg_df
    .filter(F.col("상권업종대분류명") == "음식")
    .select(
    F.col("상호명"),
    F.col("지점명"),
    F.col("상권업종대분류명"),
    F.col("상권업종중분류명"),
    F.col("상권업종소분류명"),
    F.col("지번코드"),
    F.col("경도").cast("double"),
    F.col("위도").cast("double")
    )
)
restaurant_gg_df.printSchema()
print(restaurant_gg_df.count())
restaurant_gg_df.show(5, truncate=False)

root
 |-- 상호명: string (nullable = true)
 |-- 지점명: string (nullable = true)
 |-- 상권업종대분류명: string (nullable = true)
 |-- 상권업종중분류명: string (nullable = true)
 |-- 상권업종소분류명: string (nullable = true)
 |-- 지번코드: string (nullable = true)
 |-- 경도: double (nullable = true)
 |-- 위도: double (nullable = true)

4372
+--------------------+------+----------------+----------------+----------------+-------------------+----------------+----------------+
|상호명              |지점명|상권업종대분류명|상권업종중분류명|상권업종소분류명|지번코드           |경도            |위도            |
+--------------------+------+----------------+----------------+----------------+-------------------+----------------+----------------+
|전설의본곱창주희네점|NULL  |음식            |한식            |곱창 전골/구이  |4146110500103130001|127.21429642246 |37.2590061529013|
|철이네밥집포차      |NULL  |음식            |기타 간이       |김밥/만두/분식  |4146110600103930001|127.225210709652|37.252174021369 |
|두남자이야기        |NULL  |음식            |중식            |중국집          |4146110600105050007|127.222933

# 1

## 1-a

In [11]:
t = toji_gg_df.alias("t")
b = building_gg_df.alias("b")

toji_building_gg_df = (
    t.join(b, F.col("t.고유번호") == F.col("b.고유번호"), "left")
     .drop(F.col("b.고유번호"))
)

toji_building_gg_df.printSchema()
print(toji_building_gg_df.count())
toji_building_gg_df.show(5, truncate=False)

root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동원인: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 공시지가: long (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)



117096


[Stage 41:===========================================>              (3 + 1) / 4]

+-------------------+-----------------------------+----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|고유번호           |법정동명                     |지번|소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK|옥외자주식면적|
+-------------------+-----------------------------+----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|4146110100100080001|경기도 용인시 처인구 김량장동|8-1 |성명(명칭)변경|2024-09-05    |31.41   |대  |2564000 |8   |1   |NULL              |NULL          |
|4146110100100080004|경기도 용인시 처인구 김량장동|8-4 |소유권이전    |2003-03-17    |394.0   |대  |2408000 |8   |4   |1116116027        |0             |
|4146110100100080004|경기도 용인시 처인구 김량장동|8-4 |소유권이전    |2003-03-17    |394.0   |대  |2408000 |8   |4   |1116116026        |0             |
|4146110100100080011|경기도 용인시 처인구 김량장동|8-11|소유권이전    |2025-07-14    |48.38   |대  |1495000 |8   |11  |NULL              |NULL          |
|4146110100100080013|경기도 용인시 처인구 김량장동|8-13|소유권이전   

## 1-b

In [12]:
pk_cnt_df = (
    toji_building_gg_df
    .groupBy("고유번호")
    .agg(F.count("관리_건축물대장_PK").alias("pk_cnt"))
)

toji_binary_building_gg_df = (
    toji_building_gg_df
    .join(
        pk_cnt_df.filter(F.col("pk_cnt") <= 1),
        on="고유번호",
        how="inner"
    )
    .drop("pk_cnt")
)

print(toji_binary_building_gg_df.count())
toji_binary_building_gg_df.show(5, truncate=False)

108344


+-------------------+-----------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|고유번호           |법정동명                     |지번 |소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK|옥외자주식면적|
+-------------------+-----------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|4146110100100080001|경기도 용인시 처인구 김량장동|8-1  |성명(명칭)변경|2024-09-05    |31.41   |대  |2564000 |8   |1   |NULL              |NULL          |
|4146110100100080011|경기도 용인시 처인구 김량장동|8-11 |소유권이전    |2025-07-14    |48.38   |대  |1495000 |8   |11  |NULL              |NULL          |
|4146110100100080013|경기도 용인시 처인구 김량장동|8-13 |소유권이전    |2004-08-16    |55.0    |답  |1950000 |8   |13  |NULL              |NULL          |
|4146110100100200017|경기도 용인시 처인구 김량장동|20-17|소유권이전    |2016-09-22    |44.0    |대  |2199000 |20  |17  |NULL              |NULL          |
|4146110100100220003|경기도 용인시 처인구 김량장동|22-3 |

In [13]:
toji_binary_building_gg_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), F.lit(0))
     .otherwise(F.lit(1))
).groupBy("건물개수").count().orderBy("건물개수").show()

+--------+-----+
|건물개수|count|
+--------+-----+
|       0|92580|
|       1|15764|
+--------+-----+



# 2

## 2-a

In [14]:
toji_with_1_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNotNull())
)

# 확인
print(toji_with_1_building_gg_df.count())
toji_with_1_building_gg_df.show(5, truncate=False)

15764


+-------------------+-----------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|고유번호           |법정동명                     |지번 |소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK|옥외자주식면적|
+-------------------+-----------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|4146110100100310008|경기도 용인시 처인구 김량장동|31-8 |소유권이전    |2016-02-01    |195.0   |대  |1890000 |31  |8   |11161100269304    |0             |
|4146110100100400009|경기도 용인시 처인구 김량장동|40-9 |소유권이전    |2004-06-23    |151.0   |대  |2033000 |40  |9   |1116131856        |0             |
|4146110100100630021|경기도 용인시 처인구 김량장동|63-21|소유권이전    |2002-05-24    |59.0    |대  |3036000 |63  |21  |11161100239647    |0             |
|4146110100100820002|경기도 용인시 처인구 김량장동|82-2 |소유권이전    |1967-08-24    |89.0    |대  |3102000 |82  |2   |1116136987        |0             |
|4146110100101030004|경기도 용인시 처인구 김량장동|103-4

## 2-b

In [15]:
# 건물 있는데 음식점 아닌거 거르려고 만드는 join table

t = toji_with_1_building_gg_df.alias("t")
r = restaurant_gg_df.alias("r")

toji_building_restaurant_gg_df = (
    t.join(
        r,
        F.col("t.고유번호") == F.col("r.지번코드"),
        how="left"
    )
    .drop(F.col("r.지번코드"))
)

In [16]:
toji_building_restaurant_gg_df.select(
    F.count("*").alias("전체"),
    F.sum(F.col("상호명").isNull().cast("int")).alias("상호명_NULL"),
    F.sum(F.col("상호명").isNotNull().cast("int")).alias("상호명_NOT_NULL")
).show()

+-----+-----------+---------------+
| 전체|상호명_NULL|상호명_NOT_NULL|
+-----+-----------+---------------+
|16176|      14607|           1569|
+-----+-----------+---------------+



In [17]:
toji_building_restaurant_gg_df = (
    toji_building_restaurant_gg_df
    .filter(F.col("상호명").isNotNull())
)

# 확인
print(toji_building_restaurant_gg_df.count())
toji_building_restaurant_gg_df.show(5, truncate=False)

1569


+-------------------+-----------------------------+------+--------------+--------------+--------+----+--------+----+----+------------------+--------------+----------+------+----------------+----------------+----------------+----------------+----------------+
|고유번호           |법정동명                     |지번  |소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK|옥외자주식면적|상호명    |지점명|상권업종대분류명|상권업종중분류명|상권업종소분류명|경도            |위도            |
+-------------------+-----------------------------+------+--------------+--------------+--------+----+--------+----+----+------------------+--------------+----------+------+----------------+----------------+----------------+----------------+----------------+
|4146110100101030004|경기도 용인시 처인구 김량장동|103-4 |소유권이전    |2002-05-06    |314.0   |대  |2994000 |103 |4   |1116135133        |0             |초콜릿    |NULL  |음식            |주점            |일반 유흥 주점  |127.20839028152 |37.234742592202 |
|4146110100101150006|경기도 용인시 처인구 김량장동|115-6 |소유권이전    |2025-05-13    |235.0   |대  

## 2-c

In [18]:
# 아무 건물도 없는 필지
toji_with_0_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNull())
)

# 위에서 만든 join table과 concat을 위해 column 추가
toji_with_0_building_gg_df = (
    toji_with_0_building_gg_df
    .withColumn("상호명", F.lit(None).cast("string"))
    .withColumn("지점명", F.lit(None).cast("string"))
    .withColumn("상권업종대분류명", F.lit(None).cast("string"))
    .withColumn("상권업종중분류명", F.lit(None).cast("string"))
    .withColumn("상권업종소분류명", F.lit(None).cast("string"))
    .withColumn("경도", F.lit(None).cast("double"))
    .withColumn("위도", F.lit(None).cast("double"))
)

final_toji_df = (
    toji_building_restaurant_gg_df
    .unionByName(toji_with_0_building_gg_df)
)

In [19]:
# 전체 row 수 = (건물1+음식점) + (건물0)
print("final:", final_toji_df.count())
print("건물1+음식점:", toji_building_restaurant_gg_df.count())
print("건물0:", toji_with_0_building_gg_df.count())

# 건물 개수 분포
final_toji_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), 0).otherwise(1)
).groupBy("건물개수").count().orderBy("건물개수").show()


final: 94149
건물1+음식점: 1569
건물0: 92580
+--------+-----+
|건물개수|count|
+--------+-----+
|       0|92580|
|       1| 1569|
+--------+-----+



## 2-d

In [20]:
all_null_group_df = (
    final_toji_df
    .groupBy("법정동명", "본번")
    .agg(
        F.sum(F.col("상호명").isNotNull().cast("int")).alias("non_null_cnt")
    )
    .filter(F.col("non_null_cnt") == 0)
)

# 개수
all_null_group_df.count()

36816

In [21]:
clean_final_toji_df = (
    final_toji_df
    .join(
        all_null_group_df.select("법정동명", "본번"),
        on=["법정동명", "본번"],
        how="left_anti"
    )
)

# 결과 확인
clean_final_toji_df.count()

5067

In [22]:
# 건물 개수 분포
clean_final_toji_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), 0).otherwise(1)
).groupBy("건물개수").count().orderBy("건물개수").show()

+--------+-----+
|건물개수|count|
+--------+-----+
|       0| 3498|
|       1| 1569|
+--------+-----+



# 3

## 3-a

In [24]:
out_base = "/opt/spark/data/output/silver_stage_1"

out_path = (
    f"{out_base}"
    f"/year={prev_q_year}"
    f"/month={prev_q_last_month:02d}"
)

(
    clean_final_toji_df
    .write
    .mode("overwrite")
    .parquet(out_path)
)

print("[AUTO] saved to:", out_path)

[AUTO] saved to: /opt/spark/data/output/silver_stage_1/year=2025/month=12
